<a href="https://colab.research.google.com/github/michaelconsigli/Tesi/blob/main/Speech2RIR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!git clone https://github.com/anton-jeran/Speech2RIR
%cd Speech2RIR/
!bash download_model.sh

Cloning into 'Speech2RIR'...
remote: Enumerating objects: 237, done.
remote: Counting objects: 100% (237/237), done.
remote: Compressing objects: 100% (228/228), done.
remote: Total 237 (delta 55), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (237/237), 214.23 KiB | 7.39 MiB/s, done.
Resolving deltas: 100% (55/55), done.
/content/Speech2RIR
Downloading...
From (original): https://drive.google.com/uc?id=1CcF1c9i76-MVPJ-PGoBwvaVtOUBPD57y
From (redirected): https://drive.google.com/uc?id=1CcF1c9i76-MVPJ-PGoBwvaVtOUBPD57y&confirm=t&uuid=e7f634d3-4a6c-47f2-8f72-a84f21b8213b
To: /content/Speech2RIR/checkpoint-1040000steps.pkl
100% 3.38G/3.38G [00:35<00:00, 95.5MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1TrnXjR-vrCrub_RY6kdEBqA0D4hGiIp3
From (redirected): https://drive.google.com/uc?id=1TrnXjR-vrCrub_RY6kdEBqA0D4hGiIp3&confirm=t&uuid=088836e6-6780-486b-8808-e53310b45dfe
To: /content/Speech2RIR/checkpoint-1900000steps.pkl
100% 3.38G/3.38G [00:43<


CARICO IL MODELLO:

- Importa le librerie — carica il codice di PyTorch e della rete (modello) specifico che userai.

- Istanzia il modello — crea una nuova rete (modello) usando il codice della classe Generator. Qui devi specificare le stesse dimensioni e parametri con cui la rete è stata addestrata, altrimenti i pesi non ci entreranno bene.

- Carica i pesi del modello — apre il checkpoint (file con dati salvati) e legge i pesi nella parte giusta del file (checkpoint['model']['generator']).

- Inserisci i pesi nel modello — qui è il punto critico: se le dimensioni del modello e dei pesi non combaciano esce errore.

- Metti il modello in modalità valutazione — prepara il modello per fare previsioni (usare il modello, non addestrarlo).

- Stampa messaggio di conferma.


I pesi verranno utilizzati nella funzione forward del generator, in particolare nell'encoder e nel decoder. Questi due usano pesi interni es.

*conv = torch.nn.Conv1d(in_channels=1, out_channels=1, kernel_size=3*)

- Prima di caricare pesi, i valori dei filtri (kernel) sono casuali.

- Dopo conv.load_state_dict(...) con pesi addestrati, quei valori cambiano e diventano precisi per rilevare certe caratteristiche nell'input.

- Quando passi un input x a conv(x), il risultato riflette le trasformazioni con quei pesi.

In [7]:
# 1️⃣ Import delle librerie
import torch
from models.autoencoder.AudioDec import Generator

# 2️⃣ Crea il modello (con i parametri di default)
# Parametri dal file di configurazione YAML
model = Generator(
    input_channels=1,
    output_channels_rir=1,
    encode_channels=16,
    decode_channels=16,
    code_dim=128,
    codebook_num=64,
    codebook_size=8192,
    bias=True,
    combine_enc_ratios=[],
    combine_enc_strides=[],
    seperate_enc_ratios_rir=[2, 4, 8, 12, 16, 32],
    seperate_enc_strides_rir=[2, 2, 3, 5, 5, 5],
    rir_dec_ratios=[256, 128, 64, 32, 32, 32, 16],
    rir_dec_strides=[5, 5, 2, 2, 2, 1, 1],
    mode='causal',
    codec='audiodec',
    projector='conv1d',
    quantier='residual_vq',
)

# 3️⃣ Carica i pesi pre-addestrati
checkpoint_path = "exp/autoencoder/symAD_vctk_48000_hop300/checkpoint-1900000steps.pkl"
checkpoint = torch.load(checkpoint_path, map_location='cpu')

# Carica nello stato del modello solo la parte 'generator' dentro 'model'
model.load_state_dict(checkpoint['model']['generator'])

# 4️⃣ Metti il modello in modalità "valutazione"
model.eval()

print("✅ Modello caricato correttamente!")


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 131MB/s]


✅ Modello caricato correttamente!


- carico e preparo l'audio
- stimo la RIR
- salvo

In [ ]:
import torch
import torchaudio

# 1️⃣ Carica l’audio
audio_path = "esempio.wav"
waveform, sr = torchaudio.load(audio_path)

# 2️⃣ Porta il sample rate a 48000 Hz se serve
if sr != 48000:
    waveform = torchaudio.functional.resample(waveform, sr, 48000)
    sr = 48000

# 3️⃣ Converte in mono (il modello accetta 1 canale)
if waveform.shape[0] > 1:
    waveform = torch.mean(waveform, dim=0, keepdim=True)

# 4️⃣ Aggiungi dimensione batch (B, C, T)
waveform = waveform.unsqueeze(0)

# 5️⃣ Stima la RIR
with torch.no_grad():
    estimated_rir = model(waveform)

# 6️⃣ Risultato
print("✅ RIR stimata!")
print("Shape:", estimated_rir.shape)
